In [61]:
import pysam
import pybedtools
import pandas
from sklearn.cluster import KMeans


In [118]:
class Motifs:

    """
    
    
    input here should be motifs in peaks already
    
    
    
    
    -----------------
    Attributes
    -----------------
    
    bam_file: alignment file in BAM format containing reads from an ATAC-seq experiment;
              file should be sorted and indexed.
    
    motifs_file: transcription factor motifs in open regions of the genome formatted as:
                 chromosome start end motif-ID score strand (score can be e-value, p-value, doesn't matter...)
                 tab delimited, no header, should include only motifs in peaks
    
    motifs: holds a dictionary with motifs from motifs_file;
            ---
            {TF1:[(occurence1_chrom, occurence1_start, occurence1_end, occurence1_p-value, occurence1_strand),
                  (occurence2_chrom, occurence2_start, occurence2_end, occurence2_p-value, occurence2_strand)],
             TF2:[(occurence1_chrom, occurence1_start, occurence1_end, occurence1_p-value, occurence1_strand)]...}        
            ---
            Keys: motif IDs -- will be used later to build the prior, thus I use transcription factors as IDs.
            Values: list of occurences in the genome of the motif(s) associated with identifier.

    insertion_counts: holds a dictionary with matrices of insertion counts around motifs for a particular transcription factor
                      ---
                      Keys: motif holders -- will be used later to build the prior 
                      Values: pandas DataFrame with insertion counts around each motif for the motif(s) associated with identifier,
                              rownames are motif holders -- chrom,start,end,strand.

    tn5_9mer_counts: holds a dictionary with matrices of 9mer counts around motifs for a particular transcription factor
                     ---
                     Keys: motif holders -- will be used later to build the prior
                     Values: pandas DataFrame with 9mer counts around each motif for the motif(s) associated with identifier,
                             rownames are motif holders -- chrom,start,end,strand.

    motif_clusters_insertion_counts:

    motif_clusters_9mer_counts:

    
    """

    def __init__(self, bam_file, motifs_file):
        
        self.bam_file = bam_file
        self.motifs_file = motifs_file 
        self.motifs = {}
        self.insertion_counts = {}
        self.tn5_9mer_counts = {}
        self.motif_clusters_insertion_counts = {}
        self.motif_clusters_9mer_counts = {}
        
    def read_motifs_file(self):
        
        """
        Reads in a motifs file and saves a dictionary of motifs for each transcription factor in self.motifs 
        """
        
        motifs_file = self.motifs_file

        with open(motifs_file, "r") as motifs_handle:

            for motif_occurence in motifs_handle:

                motif_occurence = motif_occurence.split()

                motif_id = motif_occurence[4]
                chrom = motif_occurence[0]
                start = motif_occurence[1]
                end = motif_occurence[2]
                strand = motif_occurence[5]
                score = motif_occurence[4]

                to_bed = [chrom, int(start), int(end), motif_id, score, strand]

                if motif_id in self.motifs:
                    self.motifs[motif_id].append(tuple(to_bed))
                else:
                    self.motifs[motif_id] = [tuple(to_bed)]
    

    @staticmethod 
    def get_insertions(bam_handle, chrom, start, end, upstream = 50, downstream = 50):
        
        """
        Returns insertion counts at a given region of the genome
        """
        

        # initialize counts vector at 0 for all positions
        region_length = (end - start) + 1
        insertion_counts = [0]*region_length

        # fetch reads mapping to the region specified as input
        reads = bam_handle.fetch(chrom, start, end)

        # each read represents a potential insertion within region
        for read in reads:
            
            if read.is_reverse: 
                # offset by 5 bp
                insertion_pos = read.reference_end - 5 
            else:
                # offset by 4 bp
                insertion_pos = read.reference_start + 4
            
            pos = insertion_pos - start 
            
            # make sure pos is within region
            if pos in range(0, region_length):
                insertion_counts[pos] += 1  
                    
        return tuple(insertion_counts)
        
    @staticmethod
    def get_tn5_9mer(insertion_counts, up_offset = 4, down_offset = 5):
        
        """
        Smooth insertion counts to Tn5 occupancy (9mer track)
        """
    
        tn5_9mer_counts = list(insertion_counts)

        region = range(0, len(insertion_counts))

        for pos in region:
            for idx in range(pos - up_offset, pos + down_offset + 1):
                if idx in region and idx != pos:
                    tn5_9mer_counts[idx] += insertion_counts[pos]
        
        return tuple(tn5_9mer_counts)
            
    
    def compute_scores_matrices(self, upstream, downstream):
        
        """
        Generate matrices of insertion and 9mer counts for all motifs in self.motifs
        """
        
        bam_handle = pysam.AlignmentFile(self.bam_file, "rb")
        
        for motif_id in self.motifs.keys(): ## Paralelize
            
            insertion_counts_mat = []
            tn5_9mer_counts_mat = []
            rownames_motifs = []
            
            motifs = pybedtools.Bedtool(self.motifs[motif_id])
            motifs = motifs.sort()
            
            for motif in motifs:
                
                # get motif coordinates -- FIMO output is a closed interval, so add 1 to end (python)
                chrom = motif.chrom
                center = round(stat.median(range(motif.start, motif.end + 1)))
                start = center - upstream
                end = center + downstream
                
                insertion_counts = get_insertions(bam_handle, chrom, start, end)
                insertion_counts_mat.append(insertion_counts)
                
                tn5_9mer_counts = get_tn5_9mer(insertion_counts)
                tn5_9mer_counts_mat.append(tn5_9mer_counts)
                
                motif_holder = ",".join([motif.chrom, str(motif.start), str(motif.end), motif.strand])
                rownames_motifs.append(motif_holder)
                
            
            insertion_counts_df = pandas.DataFrame(insertion_counts_mat, 
                                                   index = rownames_motifs)
            
            tn5_9mer_counts_df = pandas.DataFrame(tn5_9mer_counts_mat, 
                                                  index = rownames_motifs)
            
            self.insertion_counts[motif_id] = insertion_counts_df
            self.tn5_9mer_counts[motif_id] = tn5_9mer_counts_df
            

    def cluster_motifs(self):
        
        """
        Use K-Means to cluster motifs for a given transcription factor in two clusters for all TFs in self.motifs
        """
        ## Paralelize, as of now I am going to do both ins and 9mer (choose one later?)
        
        # insertion counts
        for motif_id in self.insertion_counts.keys():
            
            insertion_counts_df = self.insertion_counts[motif_id]
            insertion_counts_mat = insertion_counts_df.as_matrix()

            motif_clusters = KMeans(n_clusters = 2)
            motif_clusters.fit(insertion_counts_mat)
            labels = motif_clusters.labels_
            results = pandas.DataFrame([insertion_counts_df.index, labels]).T
            results.columns = ['motif', 'cluster']     
            
            clust0 = insertion_counts_df[insertion_counts_df.index.isin(results['motif'][results['cluster'] == 0])]
            clust1 = insertion_counts_df[insertion_counts_df.index.isin(results['motif'][results['cluster'] == 1])]
            
            clust0_mean = clust0.mean(1).mean()
            clust1_mean = clust1.mean(1).mean()
            
            if clust0_mean > clust1_mean:
                
                results['cluster'][results['cluster'] == 0] = 'keep'
                results['cluster'][results['cluster'] == 1] = 'not_keep'
            
            if clust1_mean > clust0_mean:
                
                results['cluster'][results['cluster'] == 1] = 'keep'
                results['cluster'][results['cluster'] == 0] = 'not_keep'
            
            self.motif_clusters_insertion_counts[motif_id] = results
            
            
            
        # 9mers
        for motif_id in self.tn5_9mer_counts.keys():
            
            tn5_9mer_counts_df = self.tn5_9mer_counts[motif_id]
            tn5_9mer_counts_mat = tn5_9mer_counts_df.as_matrix()

            motif_clusters = KMeans(n_clusters = 2)
            motif_clusters.fit(tn5_9mer_counts_mat)
            labels = motif_clusters.labels_
            results = pandas.DataFrame([tn5_9mer_counts_df.index, labels]).T
            results.columns = ['motif', 'cluster']
            
            clust0 = insertion_counts_df[insertion_counts_df.index.isin(results['motif'][results['cluster'] == 0])]
            clust1 = insertion_counts_df[insertion_counts_df.index.isin(results['motif'][results['cluster'] == 1])]
            
            clust0_mean = clust0.mean(1).mean()
            clust1_mean = clust1.mean(1).mean()
            
            if clust0_mean > clust1_mean:
                
                results['cluster'][results['cluster'] == 0] = 'keep'
                results['cluster'][results['cluster'] == 1] = 'not_keep'
            
            if clust1_mean > clust0_mean:
                
                results['cluster'][results['cluster'] == 1] = 'keep'
                results['cluster'][results['cluster'] == 0] = 'not_keep'
            
            self.motif_clusters_9mer_counts[motif_id] = results

            

class Edge:
    
    def __init__(self, regulator, target, weight, motif, gene):
        
        self.regulator = regulator
        self.target = target
        self.weight = weight
        self.motif = motif
        self.gene = gene
        
        
class Prior:
    
    """
    
    -----------------
    Attributes
    -----------------
    
    bam_file: aligned ATAC-seq BAM file
    
    motifs_file: motifs annotation in FIMO format IN PEAKS
    
    peaks_file: ATAC peaks in BED format (e.g. called by MACS)
    
    tss_file: TSS annotation in BED format
    
    transcription_factors:
    
    target_genes:
    
    upstream_in_matrix: 
    
    downstream_in_matrix:
    
    cluster: whether to cluster motifs or not 
    
    max_upstream_distance: bp upstream allowed for assignment peak to gene
    
    max_downstream_distance: bp downstream allowed for assignment peak to gene
        
    """
    
    def __init__(self, bam_file, motifs_file,  peaks_file, tss_file,
                 transcription_factors, target_genes, cluster = True,
                 upstream_in_matrix = 50, downstream_in_matrix = 50, 
                 max_upstream_distance = float("Inf"), max_downstream_distance = float("Inf")):

        self.bam_file = bam_file     
        self.motifs_file = motifs_file 
        self.peaks_file = peaks_file   
        self.tss_file = tss_file
        self.transcription_factors = transcription_factors
        self.target_genes = target_genes
        self.upstream_in_matrix = upstream_in_matrix
        self.downstream_in_matrix = downstream_in_matrix
        self.cluster = cluster 
        self.max_upstream_distance = max_upstream_distance     
        self.max_downstream_distance = max_downstream_distance 
        
        self.motifs = []
        self.peaks = []
        self.tss = []
        
        
    def __str__(self):
        
        """
        
        """
        
        print "Prior generated from: "
        print "Alignment file: " + self.bam_file
        print "Motifs file: " + self.motifs_file
        print "Peaks file: " + self.peaks_file
        print "TSS annotation file: " + self.tss_file
        #### add other parameters later
        
    
    
    def get_motifs(self):
        
        
        """
        returns BedTool with good motifs (?)
        """
        
        
        if self.cluster == True:
            
            motifs_keep_ins = []
            motifs_keep_9mer = []
            
            
            # class Motifs to cluster etc
            motifs = Motifs(self.bam_file, self.motifs_file)
            motifs.read_motifs_file()
            motifs.compute_scores_matrices(upstream = upstream_in_matrix, 
                                           downstream = downstream_in_matrix)
            motifs.cluster_motifs()
            
            # find out which motifs to keep
            for regulator in motifs.motif_clusters_insertion_counts.keys():
                
                keep = motifs.motif_clusters_insertion_counts[regulator]['cluster'] == 'keep'
                tmp_motifs_keep = motifs.motif_clusters_insertion_counts[regulator]['motif'][keep] 
            
                # append good motifs
                for motif in motifs_keep:
                    motifs_keep_ins.append(tuple(motif.split(',')))

            # 9mer -- after will keep only one?
            #for regulator in motifs.motif_clusters_9mer_counts.keys():
                
            #    keep = motifs.motif_clusters_9mer_counts[regulator]['cluster'] == 'keep'
            #    tmp_motifs_keep = motifs.motif_clusters_9mer_counts[regulator]['motif'][keep] 
            
                # append good motifs
            #    for motif in motifs_keep:
            #        motifs_keep_9mer.append(tuple(motif.split(',')))

            
            motifs = pybedtools.BedTool(motifs_keep_ins)
            #motifs = pybedtools.BedTool(motifs_keep_9mer)
        
        else:
            
            motifs = pybedtools.BedTools(self.motifs_file)
        
        motifs.sort()
        
        self.motifs = motifs
    
    
    def get_peaks(self):
        
        """
        
        """
        
        peaks = pybedtools.Bedtool(self.peaks_file)
        peaks = peaks.sort()
        
        self.peaks = peaks
    
    
    def get_tss(self):
        
        """
        
        """
        
        tss_genes = pybedtools.BedTool(self.tss_file)
        tss_genes = genes_tss.sort()       
        
        self.tss = tss_genes
    
    
    def assign_motifs_to_genes(self):
        
        """
        
        """
        
        motifs = self.motifs
        peaks = self.peaks
        tss = self.tss
        
        peaks_to_genes = peaks.closest(tss, D = 'b')
        motifs_to_genes = motifs.closest(peaks_to_genes)
        
        return(motifs_to_genes)
    
    
    def make_prior(self):
        
        """
        
        """
        
        self.get_motifs()
        self.get_peaks()
        self.get_tss()
        
        motifs_to_genes = self.assign_motifs_to_genes()
        
        # format prior output
        prior = set()

        # format lines in prior
        for interaction in motifs_to_genes:
            # get fields in string format
            interaction = interaction.fields
            peak = interaction[9]
            # TF name
            regulators = interaction[3]
            regulators = regulators.split('::')
            # target gene
            target = interaction[23].split()[1]
            target = target.replace('"', '').replace(';', '')
            # distance to feature, if negative it is upstream
            distance = int(interaction[24])

            for regulator in regulators:
                prior_line = (regulator.encode('ascii', 'ignore'),
                              target.encode('ascii', 'ignore'),
                              peak.encode('ascii', 'ignore'), str(distance))
                prior.add(prior_line)
                
        return(prior)
    
    
    #def write_prior(prior, outfile, outdir):

     #   prior_outdir = os.path.join(outdir, 'priors')

        #if not os.path.exists(prior_outdir):
         #   os.makedirs(prior_outdir)

        #filepath = os.path.join(prior_outdir, outfile)

        # write prior in table format regulator \t target
        #with open(filepath, 'a') as handle:
        #    handle.writelines('\t'.join(i) + '\n' for i in prior)

        #handle.close()
    

In [119]:

Motifs()
    
    

TypeError: __init__() takes exactly 3 arguments (1 given)

In [120]:
# inputs
motifs_file = './input-tests/motifs.random100lines.chr10.txt'
bam_file = './input-tests/patho_48hrs_2_dedup_mitRM_mapq30p_sorted.bam'

example = Motifs(bam_file, motifs_file)
example.read_motifs_file()
bam_handle = pysam.AlignmentFile(example.bam_file, "rb")
motif_id = example.motifs.keys()[5]
example.motifs.values()[5]


[('chr10', 10114679, 10114693, 'M5483_1.01', '1.27e-06', '+')]

In [121]:
insertion_counts_matrix = [(9,0,0,0,4,3,5), (9,0,0,0,4,3,5), (0,0,0,0,0,0,2), (0,0,0,0,0,0,2)]
rownames_motifs = ['chr10,10114679,10114693,+', 'chr10,10114679,10114694,+', 
                  'chr10,10114679,10114695,-', 'chr10,10114679,10114696,+']

insertion_counts_matrix = pandas.DataFrame(insertion_counts_matrix, index = rownames_motifs)

example.insertion_counts = {"Stat3": insertion_counts_matrix,
                            "Stat2": insertion_counts_matrix}

example.tn5_9mer_counts = {"Stat3": insertion_counts_matrix,
                            "Stat2": insertion_counts_matrix,}

In [122]:
insertion_counts_matrix.mean(1).mean()


1.6428571428571428

In [132]:
example.cluster_motifs()
example.motif_clusters_insertion_counts['Stat2']

,motif,cluster
0,"chr10,10114679,10114693,+",keep
1,"chr10,10114679,10114694,+",keep
2,"chr10,10114679,10114695,-",not_keep
3,"chr10,10114679,10114696,+",not_keep


In [137]:
for i in example.motif_clusters_insertion_counts['Stat2']['motif'][example.motif_clusters_insertion_counts['Stat2']['cluster'] == 'keep']:
    print tuple(i.split(','))

('chr10', '10114679', '10114693', '+')
('chr10', '10114679', '10114694', '+')


In [124]:
df = example.insertion_counts['Stat2'] 
df

,0,1,2,3,4,5,6
"chr10,10114679,10114693,+",9,0,0,0,4,3,5
"chr10,10114679,10114694,+",9,0,0,0,4,3,5
"chr10,10114679,10114695,-",0,0,0,0,0,0,2
"chr10,10114679,10114696,+",0,0,0,0,0,0,2


In [125]:
clust = example.motif_clusters_9mer_counts['Stat2']
#clust.query('clust.motif == 0')


#df[(df.index in example.motif_clusters_9mer_counts['Stat2']['motif'][example.motif_clusters_9mer_counts['Stat2']['cluster'] == 0])

In [126]:
df[df.index.isin(clust['motif'][clust['cluster'] == 0])]

,0,1,2,3,4,5,6
"chr10,10114679,10114695,-",0,0,0,0,0,0,2
"chr10,10114679,10114696,+",0,0,0,0,0,0,2


In [127]:
chrom = 'chr1'
start = 20784345
end = 20784360
Motifs.get_insertions(bam_handle, chrom, start, end)




(1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0)

In [128]:
insertion_counts_matrix

,0,1,2,3,4,5,6
"chr10,10114679,10114693,+",9,0,0,0,4,3,5
"chr10,10114679,10114694,+",9,0,0,0,4,3,5
"chr10,10114679,10114695,-",0,0,0,0,0,0,2
"chr10,10114679,10114696,+",0,0,0,0,0,0,2


In [129]:
dataset_array = insertion_counts_matrix.values
print(dataset_array.dtype)
print(dataset_array)




int64
[[9 0 0 0 4 3 5]
 [9 0 0 0 4 3 5]
 [0 0 0 0 0 0 2]
 [0 0 0 0 0 0 2]]


In [13]:
dataset = insertion_counts_matrix

# Convert DataFrame to matrix
mat = dataset.as_matrix()
# Using sklearn
km = KMeans(n_clusters = 2)
km.fit(mat)
# Get cluster assignment labels
labels = km.labels_
# Format results as a DataFrame
results = pandas.DataFrame([dataset.index,labels]).T

In [14]:
pandas.DataFrame([dataset.index,labels])

,0,1,2,3
0,"chr10,10114679,10114693,M5483_1.01","chr10,10114679,10114693,M5483_1.02","chr10,10114679,10114693,M5483_1.03","chr10,10114679,10114693,M5483_1.04"
1,1,0,0,0


In [15]:
dataset

,0,1,2,3,4,5,6
"chr10,10114679,10114693,M5483_1.01",9,0,0,0,4,3,5
"chr10,10114679,10114693,M5483_1.02",1,0,0,4,2,1,0
"chr10,10114679,10114693,M5483_1.03",0,1,0,0,0,0,2
"chr10,10114679,10114693,M5483_1.04",0,0,0,0,4,1,5


In [16]:
dataset.index

Index([u'chr10,10114679,10114693,M5483_1.01',
       u'chr10,10114679,10114693,M5483_1.02',
       u'chr10,10114679,10114693,M5483_1.03',
       u'chr10,10114679,10114693,M5483_1.04'],
      dtype='object')

In [17]:
results

,0,1
0,"chr10,10114679,10114693,M5483_1.01",1
1,"chr10,10114679,10114693,M5483_1.02",0
2,"chr10,10114679,10114693,M5483_1.03",0
3,"chr10,10114679,10114693,M5483_1.04",0


In [ ]:


#### GENERATING THE PRIOR...

        #
        motifs = pybedtools.Bedtool(self.motifs_bed)
        motifs = motifs.sort()

        #
        peaks = pybedtools.Bedtool(self.peaks_bed)
        peaks = peaks.sort()

        #
        genes_tss = pybedtools.BedTool(self.tss_bed)
        genes_tss = genes_tss.sort()

        # assign peaks to closes feature in feature file
        peaks_to_genes = peaks.closest(genes_tss, D = 'b')
        motifs_to_genes = motifs.closest(peaks_to_genes)
    